In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

pd.set_option('display.max_columns', 500)

# Load the data
flights_with_products = pd.read_parquet("../data/flights_with_products.parquet")
# flights_with_products.head()

In [13]:
flights_data = flights_with_products.dropna()


In [14]:
flights_data.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Agua Natural 600 Ml,Aifa A Central Del Norte,Aifa A Central Taxqueã±a,Akumal O Puerto Aventura Sencillo 6 Pax,Akumal O Puerto Aventuras Redondo 12 Pax,Akumal O Puerto Aventuras Redondo 6 Pax,Akumal O Puerto Aventuras Sencillo 12 Pax,Amstel Ultra,Apto Cun Puerto-Morelos-Redondo 1 A 6 Paxs,Apto Cun Puerto-Morelos-Redondo7 A 12,Apto Cun Puerto-Morelos-Sencillo 1 A 6 Paxs,Apto Cun Puerto-Morelos-Sencillo 7 A 12,Apto Cun-Centro/Hoteles/Pto Juarez 1 A 6 Redondo,Apto Cun-Centro/Hoteles/Pto Juarez 1 A 6 Sencillo,Apto Cun-Centro/Hoteles/Pto Juarez 12paxs Sencillo,Apto Cun-Centro/Hoteles/Pto Juarez 7 A 12 Redondo,Apto.Gdl.-Terminal Zapopan,Arandano,Arandano Mango Mix,Arcoiris,Baileys,Baileys,Botana Sabritas Con Dip De Queso,Cafe 19 Cafe Clasico,Cafe 19 Capuchino,Cafe 19 Chiapas,Cafe Costa,Cafe De Olla,Cancun Plaza Las Americas Sencillo,Capitan Morning,Capitan Morning Con Pan Dulce,Carne Seca Habanero,Carne Seca Original,Cat.A.Subc,Cat.C.Com,Cat.D.Sdn,Cat.L.,Cerveza Charter,Charter Cheve Doble,Charter Licor Doble,Cheetos,Cheetos Flamin Hot,Cheve+Carne+Hsbc,Chokis,Ciel Mineralizada,Club Sandwich,Coca Cola Dieta,Coca Cola Regular,Coca Sin Azucar,Coco Bongo Cancun Apartado,Coco Bongo Full Pack,Coco Bongo Playa,Corajillo,Corajillo Baileys,Cuerno Clasico De Pavo,Cuerno Individual Charter,Dip De Queso,Doritos Nacho,Eco Holder,Emperador Chocolate,Emperador Vainilla,Fanta De Naranja,Fritos Limon Y Sal,Frutos Secos Enchilados,Galleta De Arandano Relleno De Q/Crema,Galleta De Chispas De Chocolate,Galleta De Chocolate,Go Nuts,Gomita Enchilada La Cueva,Guanajuato :Bjx A Central De Autobuses,Hazme Doble,Heineken 0,Heineken Original,Heineken Silver,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres Priv-Redondo 1,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres Priv-Redondo 6,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres-Priv-Sencillo,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujerespriv-Sencillo 1,Hsbc Promo 1c,Hsbc-Viva,Jack And Coke,Jugo De Mango,Jugo De Manzana,Jw Red Label,Jw Red Label,Kacang Flaming Hot,Leche De Chocolate Sc,Leche De Fresa Sc,Licor + Refresco,Licor Charter,Luxury Nut Mix,Mafer Sin Sal,Mega Cuerno Clasico,Mega Cuerno Tripulacion,Muffin Integral,Nishikawa Japones,Nishikawa Salado,Nissin Dark Dragon,Nissin Fuego,Nissin Limon Y Habanero,Nissin Picante,Nissin Res,Nueces De Arbol Mix,Nutty Berry Mix,Panini Clasico,Panini Integral,Promo Amstel,Promo Hsbc 1 Bebida Gratis,Protein Adventure,Quaker Avena Frutos Rojos,Quaker Avena Moras,Quaker Granola,Quaker Natural Balance,Rancheritos,Ron Bacardi,Ruffles Queso,Sabritas Flamin Hot,Sabritas Originales,Salsa Botanera,San Miguel De Allende:Bjx A Central De Autobuses,Sidral Mundet,Silao:Bjx A Central De Autobuses,Sol Clamato,Sprite,Taxi Acapulco Zona Dorada 3 Pax,Taxi Cabos Zona 1 4pax,Taxi Cabos Zona 2 4pax,Taxi Cabos Zona 3 4pax,Taxi Cdmx - Aeropuerto,Taxi Cdmx - Centro.,Taxi Cdmx - Observatorio,Taxi Cdmx - Santa Fe,Taxi Cdmx - Sur. Norte,Taxi Cdmx A Polanco-Ãngel Independencia.,Taxi Cdmx- Interlomas_Bosque-Real,Taxi Huatulco Mazunte/Zipolite 3pax,Taxi Huatulco Puerto Escondido 3pax,Taxi Tlc: Central. San Mateo,Taxi Tlc: Centro,Taxi Tlc: Robles. Ocoyoacac,Taxi Tlc: Sendero,Taxi Tlc: Zona Industial,Taxi Veracruz-Boca Rão-4pax,Taxi Veracruz-Boca Rão-6pax,Taxi Zihuatanejo Zona 1-3pax,Taxi Zihuatanejo Zona 2 - 3pax,Te Frutos Rojos,Te Manzanilla Jengibre,Te Relax,Te Vainilla,Tecate Light,Tequila + Mezclador,Tequila 7 Leguas Blanco,Tequila 7 Leguas Reposado,Tinto,Topochico Seltzer Fresa-Guayaba,Topochico Seltzer Mango,Tostitos,Tostitos Nachos Con Dip,Transfer Cjs : Aeropuerto De El Paso,Transfer Cjs: Aero. Cd. Juã¡rez,Transfer Cjs: Centro De El Paso,Transfer Cjs: Consulado Americano,Transfer Cun: Zona Hotelera Sencillo,Transfer Leon: Central.Puerta Milenio Mega Centro.Poliforum,Transfer Tlc: Aeropuerto A Cuautitlã¡n,Transfer Tlc: Aeropuerto A Observato

In [15]:
# Convertir las columnas de fecha y hora
flights_data['STD'] = pd.to_datetime(flights_data['STD'])
flights_data['STA'] = pd.to_datetime(flights_data['STA'])

# Extraer características de fecha y hora
flights_data['STD_hour'] = flights_data['STD'].dt.hour
flights_data['STA_hour'] = flights_data['STA'].dt.hour
flights_data['STD_day_of_week'] = flights_data['STD'].dt.dayofweek
flights_data['STA_day_of_week'] = flights_data['STA'].dt.dayofweek

In [27]:
x_columns = ['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'Capacity', 'Passengers', 'Bookings', 'STD_hour', 'STA_hour', 'STD_day_of_week', 'STA_day_of_week']
y_columns = [col for col in flights_data.columns if col not in x_columns + ['Flight_ID'] + ['STD'] + ['STA']]


In [28]:
X = flights_data[x_columns]
y = flights_data[y_columns]

In [29]:
categorical_features = ['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type']
numeric_features = ['Capacity', 'Passengers', 'Bookings', 'STD_hour', 'STA_hour', 'STD_day_of_week', 'STA_day_of_week']


In [30]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [31]:
# Modelo de regresión para múltiples salidas
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, verbose=1))


In [32]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [33]:
X

,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity,Passengers,Bookings,STD_hour,STA_hour,STD_day_of_week,STA_day_of_week
0,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,240,229.0,157.0,11,14,3,3
1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,186,197.0,109.0,0,4,0,0
2,XA-VAP,BA,AB,Playa,Ciudad Principal,186,162.0,90.0,8,9,4,4
3,XA-VAP,AW,BA,Ciudad Principal,Ciudad Principal,186,157.0,110.0,6,8,4,4
4,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,240,183.0,125.0,17,18,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
106457,XA-VAE,AT,BT,Playa,Ciudad Principal,186,191.0,119.0,12,16,2,2
106458,XA-VYF,AO,BT,Playa,Ciudad Principal,180,166.0,89.0,6,10,1,1
106459,XA-VAI,AW,BT,Playa,Ciudad Principal,180,142.0,54.0,10,13,3,3
106460,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,240,189.0,86.0,7,14,4,4


In [34]:
y

,Agua Natural 600 Ml,Aifa A Central Del Norte,Aifa A Central Taxqueã±a,Akumal O Puerto Aventura Sencillo 6 Pax,Akumal O Puerto Aventuras Redondo 12 Pax,Akumal O Puerto Aventuras Redondo 6 Pax,Akumal O Puerto Aventuras Sencillo 12 Pax,Amstel Ultra,Apto Cun Puerto-Morelos-Redondo 1 A 6 Paxs,Apto Cun Puerto-Morelos-Redondo7 A 12,Apto Cun Puerto-Morelos-Sencillo 1 A 6 Paxs,Apto Cun Puerto-Morelos-Sencillo 7 A 12,Apto Cun-Centro/Hoteles/Pto Juarez 1 A 6 Redondo,Apto Cun-Centro/Hoteles/Pto Juarez 1 A 6 Sencillo,Apto Cun-Centro/Hoteles/Pto Juarez 12paxs Sencillo,Apto Cun-Centro/Hoteles/Pto Juarez 7 A 12 Redondo,Apto.Gdl.-Terminal Zapopan,Arandano,Arandano Mango Mix,Arcoiris,Baileys,Baileys,Botana Sabritas Con Dip De Queso,Cafe 19 Cafe Clasico,Cafe 19 Capuchino,Cafe 19 Chiapas,Cafe Costa,Cafe De Olla,Cancun Plaza Las Americas Sencillo,Capitan Morning,Capitan Morning Con Pan Dulce,Carne Seca Habanero,Carne Seca Original,Cat.A.Subc,Cat.C.Com,Cat.D.Sdn,Cat.L.,Cerveza Charter,Charter Cheve Doble,Charter Licor Doble,Cheetos,Cheetos Flamin Hot,Cheve+Carne+Hsbc,Chokis,Ciel Mineralizada,Club Sandwich,Coca Cola Dieta,Coca Cola Regular,Coca Sin Azucar,Coco Bongo Cancun Apartado,Coco Bongo Full Pack,Coco Bongo Playa,Corajillo,Corajillo Baileys,Cuerno Clasico De Pavo,Cuerno Individual Charter,Dip De Queso,Doritos Nacho,Eco Holder,Emperador Chocolate,Emperador Vainilla,Fanta De Naranja,Fritos Limon Y Sal,Frutos Secos Enchilados,Galleta De Arandano Relleno De Q/Crema,Galleta De Chispas De Chocolate,Galleta De Chocolate,Go Nuts,Gomita Enchilada La Cueva,Guanajuato :Bjx A Central De Autobuses,Hazme Doble,Heineken 0,Heineken Original,Heineken Silver,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres Priv-Redondo 1,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres Priv-Redondo 6,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujeres-Priv-Sencillo,Hoteles Pdc . Playa Car . Costa Mujeres O Playa Mujerespriv-Sencillo 1,Hsbc Promo 1c,Hsbc-Viva,Jack And Coke,Jugo De Mango,Jugo De Manzana,Jw Red Label,Jw Red Label,Kacang Flaming Hot,Leche De Chocolate Sc,Leche De Fresa Sc,Licor + Refresco,Licor Charter,Luxury Nut Mix,Mafer Sin Sal,Mega Cuerno Clasico,Mega Cuerno Tripulacion,Muffin Integral,Nishikawa Japones,Nishikawa Salado,Nissin Dark Dragon,Nissin Fuego,Nissin Limon Y Habanero,Nissin Picante,Nissin Res,Nueces De Arbol Mix,Nutty Berry Mix,Panini Clasico,Panini Integral,Promo Amstel,Promo Hsbc 1 Bebida Gratis,Protein Adventure,Quaker Avena Frutos Rojos,Quaker Avena Moras,Quaker Granola,Quaker Natural Balance,Rancheritos,Ron Bacardi,Ruffles Queso,Sabritas Flamin Hot,Sabritas Originales,Salsa Botanera,San Miguel De Allende:Bjx A Central De Autobuses,Sidral Mundet,Silao:Bjx A Central De Autobuses,Sol Clamato,Sprite,Taxi Acapulco Zona Dorada 3 Pax,Taxi Cabos Zona 1 4pax,Taxi Cabos Zona 2 4pax,Taxi Cabos Zona 3 4pax,Taxi Cdmx - Aeropuerto,Taxi Cdmx - Centro.,Taxi Cdmx - Observatorio,Taxi Cdmx - Santa Fe,Taxi Cdmx - Sur. Norte,Taxi Cdmx A Polanco-Ãngel Independencia.,Taxi Cdmx- Interlomas_Bosque-Real,Taxi Huatulco Mazunte/Zipolite 3pax,Taxi Huatulco Puerto Escondido 3pax,Taxi Tlc: Central. San Mateo,Taxi Tlc: Centro,Taxi Tlc: Robles. Ocoyoacac,Taxi Tlc: Sendero,Taxi Tlc: Zona Industial,Taxi Veracruz-Boca Rão-4pax,Taxi Veracruz-Boca Rão-6pax,Taxi Zihuatanejo Zona 1-3pax,Taxi Zihuatanejo Zona 2 - 3pax,Te Frutos Rojos,Te Manzanilla Jengibre,Te Relax,Te Vainilla,Tecate Light,Tequila + Mezclador,Tequila 7 Leguas Blanco,Tequila 7 Leguas Reposado,Tinto,Topochico Seltzer Fresa-Guayaba,Topochico Seltzer Mango,Tostitos,Tostitos Nachos Con Dip,Transfer Cjs : Aeropuerto De El Paso,Transfer Cjs: Aero. Cd. Juã¡rez,Transfer Cjs: Centro De El Paso,Transfer Cjs: Consulado Americano,Transfer Cun: Zona Hotelera Sencillo,Transfer Leon: Central.Puerta Milenio Mega Centro.Poliforum,Transfer Tlc: Aeropuerto A Cuautitlã¡n,Transfer Tlc: Aeropuerto A Observatorio,Transfer Tlc: Aeropuerto A Tollocan,Transfer Tlc: Tollocan A Aicm,Transfer Tlc: Tollocan A Santa Fe,Tulum Redondo

In [35]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
# Entrenamiento del modelo

pipeline.fit(X_train, y_train)
